In [1]:
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
import pandas as pd

data= pd.read_csv("data9.csv", dtype=float)
data=data.dropna(axis=0,how='any')
data.phreat[data['phreat']=='-0']=0

from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2)
print(len(train_set), "train +", len(test_set), "test")

C:\Users\Zikei\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


128716 train + 32179 test


In [3]:
train_set.head()

,x,y,d1prec,d2prec,prec,d1ndvi,d2ndvi,ndvi,d1temp,d2temp,...,d2sat2,sat2,d1sat3,d2sat3,sat3,d1sat4,d2sat4,sat4,gwlvl,truth
86881,374.0,226.0,0.3579,5.0365,11.3205,-6.9401,581.8333,3785.0,0.0175,0.3377,...,-7.7735,-224.6389,2602.0,-28.4794,-300.2500,2783.5,-6.1192,-246.1944,58.1927,-0.0356
2815,12.0,62.0,0.0903,0.4562,16.2615,12.9247,174.3333,2497.0,0.0406,0.6040,...,1.4102,69.5278,1933.0,1.0335,-38.4722,2573.5,7.7750,132.6111,29.0500,-0.0012
74333,320.0,186.0,0.3204,2.6022,10.5640,5.7564,-10.1111,983.0,0.0080,0.2680,...,10.3029,102.3056,2850.0,12.4370,132.5833,3246.5,14.9231,301.0833,84.9648,-0.0254
78641,338.0,450.0,0.0739,-1.9883,13.9865,15.7853,143.8889,1679.0,0.0167,0.3732,...,-3.5660,-43.0278,2674.0,0.8013,19.0278,3206.0,3.5356,74.8889,171.6733,-0.5520
146488,630.0,656.0,-0.0243,-4.1178,7.3040,-2.0258,-113.2778,1132.0,0.0158,0.8025,...,7.5134,116.4722,2572.0,5.6078,88.2778,2898.0,0.8447,140.0833,117.0159,0.0123


In [4]:
#X_training = train_set.drop(['phreat','x','y','d1ndwi','d2ndwi','d1evi','d2evi','d1prec','d2prec','d1ndvi','d2ndvi','d1temp','d2temp','gwlvl','t1','t2','d1sat0','d2sat0','d1sat1','d2sat1','d1sat2','d2sat2','d1sat3','d2sat3','d1sat4','d2sat4'], axis=1)
X_training = train_set.drop(['phreat','x','y'],axis=1)
Y_training = train_set [['phreat']].values

#X_testing = test_set.drop(['phreat','x','y','d1ndwi','d2ndwi','d1evi','d2evi','d1prec','d2prec','d1ndvi','d2ndvi','d1temp','d2temp','gwlvl','t1','t2','d1sat0','d2sat0','d1sat1','d2sat1','d1sat2','d2sat2','d1sat3','d2sat3','d1sat4','d2sat4'], axis=1)
X_testing = test_set.drop(['phreat','x','y'],axis=1)
Y_testing = test_set [['phreat']].values

#X_final = data.drop(['phreat','x','y','d1ndwi','d2ndwi','d1evi','d2evi','d1prec','d2prec','d1ndvi','d2ndvi','d1temp','d2temp','gwlvl','t1','t2','d1sat0','d2sat0','d1sat1','d2sat1','d1sat2','d2sat2','d1sat3','d2sat3','d1sat4','d2sat4'], axis=1)
X_final = data.drop(['phreat','x','y'],axis=1)
Y_final = data [['phreat']].values

from sklearn.preprocessing import MinMaxScaler
# All data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well. Create scalers for the inputs and outputs.
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

# It's very important that the training and test data are scaled with the same scaler.
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

X_scaled_final = X_scaler.transform(X_final)
Y_scaled_final = Y_scaler.transform(Y_final)

In [5]:
print(X_scaled_training.shape)
print(Y_scaled_training.shape)

print(X_scaled_testing.shape)
print(Y_scaled_testing.shape)


print(X_scaled_final.shape)
print(Y_scaled_final.shape)

print("Note: Y values were scaled by multiplying by {:.10f} and adding {:.4f}".format(Y_scaler.scale_[0], Y_scaler.min_[0]))

(128716, 29)
(128716, 1)
(32179, 29)
(32179, 1)
(160895, 29)
(160895, 1)
Note: Y values were scaled by multiplying by 0.0022438313 and adding 0.2196


In [6]:
print(X_scaled_training)

[[0.73624776 0.7718208  0.0739537  ... 0.38816272 0.12967194 0.61710488]
 [0.66677916 0.69671594 0.12885644 ... 0.40389401 0.09013452 0.61967952]
 [0.72651281 0.73190469 0.06554772 ... 0.41089043 0.16599321 0.61786829]
 ...
 [0.65784897 0.62880788 0.1041441  ... 0.39995801 0.39563265 0.61875145]
 [0.58908128 0.60185061 0.08145963 ... 0.40375788 0.2375877  0.62183503]
 [0.62319254 0.61294998 0.0864821  ... 0.40604657 0.38525863 0.62990323]]


In [7]:
X_training.head()

,d1prec,d2prec,prec,d1ndvi,d2ndvi,ndvi,d1temp,d2temp,temp,evapot,...,d2sat2,sat2,d1sat3,d2sat3,sat3,d1sat4,d2sat4,sat4,gwlvl,truth
86881,0.3579,5.0365,11.3205,-6.9401,581.8333,3785.0,0.0175,0.3377,11.4500,3.1640,...,-7.7735,-224.6389,2602.0,-28.4794,-300.2500,2783.5,-6.1192,-246.1944,58.1927,-0.0356
2815,0.0903,0.4562,16.2615,12.9247,174.3333,2497.0,0.0406,0.6040,6.2135,6.0879,...,1.4102,69.5278,1933.0,1.0335,-38.4722,2573.5,7.7750,132.6111,29.0500,-0.0012
74333,0.3204,2.6022,10.5640,5.7564,-10.1111,983.0,0.0080,0.2680,10.8595,2.8543,...,10.3029,102.3056,2850.0,12.4370,132.5833,3246.5,14.9231,301.0833,84.9648,-0.0254
78641,0.0739,-1.9883,13.9865,15.7853,143.8889,1679.0,0.0167,0.3732,8.6485,6.3081,...,-3.5660,-43.0278,2674.0,0.8013,19.0278,3206.0,3.5356,74.8889,171.6733,-0.5520
146488,-0.0243,-4.1178,7.3040,-2.0258,-113.2778,1132.0,0.0158,0.8025,11.1185,4.0409,...,7.5134,116.4722,2572.0,5.6078,88.2778,2898.0,0.8447,140.0833,117.0159,0.0123


In [8]:
import tensorflow as tf
# Define model parameters
learning_rate = 0.01
training_epochs = 2000
display_step = 100

# Define how many inputs and outputs are in our neural network
number_of_inputs =29
number_of_outputs =1

# Define how many neurons we want in each layer of our neural network
layer_1_nodes =100
layer_2_nodes =500
layer_3_nodes =500
layer_4_nodes =100

In [9]:
# Input Layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs))

In [10]:
# Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable(name="weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)
    
    # Layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Layer 4
with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name="weights4", shape=[layer_3_nodes, layer_4_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[layer_4_nodes], initializer=tf.zeros_initializer())
    layer_4_output = tf.nn.relu(tf.matmul(layer_3_output, weights) + biases)
    
# Output Layer
with tf.variable_scope('output'):
    weights = tf.get_variable(name="weights5", shape=[layer_4_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases5", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_4_output, weights) + biases


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
# Section Two: Define the cost function of the neural network that will measure prediction accuracy during training

with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
#    cost = tf.reduce_sum(tf.square(tf.subtract(prediction, Y)))
#    cross_entropy = -tf.reduce_sum(Y*tf.log(prediction))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))
#with tf.variable_scope('accuracy'):
 #   correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(Y,1))
  #  accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [12]:
# Section Three: Define the optimizer function that will be run to optimize the neural network

with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [13]:
# Create a summary operation to log the progress of the network
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

In [14]:
saver = tf.train.Saver()

In [15]:
# Initialize a session so that we can run TensorFlow operations
with tf.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())

    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    #training_writer = tf.summary.FileWriter('./logs/training', session.graph)
    #testing_writer = tf.summary.FileWriter('./logs/testing', session.graph)

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_scaled_training, Y: Y_scaled_training})

        # Every 100 training steps, log our progress
        if epoch % 100 == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_scaled_training, Y:Y_scaled_training})
            testing_cost, testing_summary = session.run([cost, summary], feed_dict={X: X_scaled_testing, Y:Y_scaled_testing})
            #testing_accuracy = session.run(accuracy, feed_dict={X: X_scaled_testing, Y:Y_scaled_testing})
        # Write the current training status to the log files (Which we can view with TensorBoard)
            #training_writer.add_summary(training_summary, epoch)
            #testing_writer.add_summary(testing_summary, epoch)
            
            # Print the current training status to the screen
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))
            #print("Epoch: {} -  Accuracy:{}".format(epoch, testing_accuracy))

    # Training is now complete!

    # Get the final accuracy scores by running the "cost" operation on the training and test data sets
    final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
    final_testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})
    #final_test_acc= session.run(accuracy, feed_dict={X: X_scaled_final, Y: Y_scaled_final})
    
    #print("test accuracy",final_test_acc)
    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))

    # Now that the neural network is trained, let's use it to make predictions for our test data.
    
    
    # Now that the neural network is trained, let's use it to make predictions for our test data
    # Pass in the X testing data and run the "prediciton" operation
    Y_predicted_test_scaled = session.run(prediction, feed_dict={X: X_scaled_testing})

    # Unscale the data back to it's original units (dollars)
    Y_test_predicted = Y_scaler.inverse_transform(Y_predicted_test_scaled)
    
    # Now that the neural network is trained, let's use it to make predictions for our test data
    # Pass in the X testing data and run the "prediciton" operation
    Y_predicted_train_scaled = session.run(prediction, feed_dict={X: X_scaled_training})

    # Unscale the data back to it's original units (dollars)
    Y_train_predicted = Y_scaler.inverse_transform(Y_predicted_train_scaled)
    
    
    # Pass in the X testing data and run the "prediciton" operation
    Y_predicted_scaled = session.run(prediction, feed_dict={X: X_scaled_final})

    # Unscale the data back to it's original units (dollars)
    Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)


Epoch: 0 - Training Cost: 36.5207405090332  Testing Cost: 36.559776306152344


KeyboardInterrupt: 

In [ ]:
for i in range(100):
    print( "Actual phreatophyte",data['phreat'].values[i],"  Predicted",Y_predicted[i][0])

In [ ]:
Y_final=Y_final.flatten()
data.plot(kind="scatter",x="y",y="x",alpha=0.4,c=Y_final,figsize=(10,7))

In [ ]:
Predicted=Y_predicted.flatten()
data.plot(kind="scatter",x="y",y="x",alpha=0.4,c=Predicted,figsize=(10,7))

In [ ]:
from sklearn.metrics import mean_squared_error

np.sqrt(mean_squared_error(Predicted,Y_final))/np.std(Y_final)

In [ ]:
Predicted_Train=Y_train_predicted.flatten()
np.sqrt(mean_squared_error(Predicted_Train,Y_training))/np.std(Y_training)

In [ ]:
Predicted_Test=Y_test_predicted.flatten()
np.sqrt(mean_squared_error(Predicted_Test,Y_testing))/np.std(Y_testing)